# Testing ideas for distributing geogroup applications to geokeys using integer values


Assume we have an optimised set of ads served (impression) to the user on a given device at the **(days,campaign,geogroups)** level - how do we assign this at the **postcode (geokey)** level. The geogroup can be thought of as a **Postcode Sector**.

Notebook contains some ideas for distributing geogroup (Postal sector) to geokeys (postcode) as integer values. Essentially an **integer optimisation problem**.

Some assumptions:

* There are around **1.4 million postcode geokeys**
* Around **10k** postal sectors (geo groups).
* We cannot have **fractional impressions**, so have consider approaches such as the **D'hondt method**.


# Hagenbach-Bischoff quota (Adapted D'Hondt method)

In [11]:
import numpy as np 
#initial implementation taken from the https://stackoverflow.com/questions/16226991/allocate-an-array-of-integers-proportionally-compensating-for-rounding-errors
#and adapted to suit example above. 
def hbischoff_distribute_geogroup_geokey(total_geogroup_impressions: int,target:np.ndarray,geokey_weights:np.ndarray)-> np.ndarray:
    """assign total geogroup impressions proportionally to geokeys using Hagenbach-Bischoff quota
    :total_geogroup_impressions (int): Total number of impressions at the geogroup level to distribute
    :target(np.ndarray): numpy array with the geogroup to geokey target
    :geokey_weights (np.ndarray): numpy array of geokey weights
    :return: a numpy array of integer impressions for the geokey level breakdown
    """

    assert np.sum(geokey_weights)!=0, f"issue with geokey_weights sum to 0: {geokey_weights}"

    #replace any nan values in the weights
    geokey_weights[np.isnan(geokey_weights)] = 0
    #only use the targeted geokeys
    geokey_weights = target*geokey_weights
    #normalise the weights
    geokey_weights /= geokey_weights.sum()
    

    quota=sum(geokey_weights)/(1.+total_geogroup_impressions) #force float
    frac=[geoweight/quota for geoweight in geokey_weights]
    #print("v1 frac = ",frac)

    res=[int(f) for f in frac]
    #print("res v1 ",res)
    n=total_geogroup_impressions-sum(res) #number of geokeys remaining to allocate
    #print("type n = ",n)

    if n==0: return res #done
    if n<0: return [min(x,total_geogroup_impressions) for x in res] #to handle case where geokey_weights=[0,0,..,1,0,...,0]

    #distribute the remaining impressions to the geokeys with the largest remainder
    remainders=[ai-bi for ai,bi in zip(frac,res)]

    limit=sorted(remainders,reverse=True)[n-1]
    #assign the remainder to the geokeys with the largest remainder 
    for i,r in enumerate(remainders):
        if r>=limit:
            res[i]+=1
            n-=1 # attempt to handle perfect equality
            if n==0: return res #done
    raise #should never happen

# Hagenbach-Bischoff quota (Adapted D'Hondt method) - Improved implementation

In [12]:
import numpy as np 
#initial implementation taken from the https://stackoverflow.com/questions/16226991/allocate-an-array-of-integers-proportionally-compensating-for-rounding-errors
#and adapted to suit example above. 
def hbischoff_distribute_geogroup_geokey_v2(total_geogroup_impressions: int,target:np.ndarray,geokey_weights:np.ndarray)-> np.ndarray:
    """
    assign total geogroup impressions proportionally to geokeys using Hagenbach-Bischoff quota
    :total_geogroup_impressions (int): Total number of impressions at the geogroup level to distribute
    :target(np.ndarray): numpy array with the geogroup to geokey target
    :geokey_weights (np.ndarray): numpy array of geokey weights
    :return: a numpy array of integer impressions for the geokey level breakdown
    """

    assert np.sum(geokey_weights)!=0, f"issue with geokey_weights sum to 0: {geokey_weights}"

    #replace any nan values in the weights
    geokey_weights[np.isnan(geokey_weights)] = 0
    #only use the targeted geokeys
    geokey_weights = target*geokey_weights
    #normalise the weights
    geokey_weights /= geokey_weights.sum()
    
    quota = sum(geokey_weights)/(1+total_geogroup_impressions)
    frac = [geoweight/quota for geoweight in geokey_weights]
    res = np.floor(frac).astype(int)
    #print("v2 frac = ",frac)
    #print("type = ",type(frac))
    #print("res v2= ",res," ",frac)
    n=total_geogroup_impressions-sum(res) #number of geokeys remaining to allocate
    #print("type n v2= ",n)

    if n==0: return res #done
    if n<0: return [min(x,total_geogroup_impressions) for x in res] #to handle case where geokey_weights=[0,0,..,1,0,...,0]

    #distribute the remaining impressions to the geokeys with the largest remainder
    remainders = frac - res
    
    #sort by the largest limit
    limit = remainders[np.argsort(remainders)[::-1][n-1]]

    #assign the remainder to the geokeys with the largest remainder 
    for i,r in enumerate(remainders):
        if r>=limit:
            res[i]+=1
            n-=1 # attempt to handle perfect equality
            if n==0: return res #done
    raise #should never happen

def hbischoff_distribute_geogroup_base(total_geogroup_impressions: int,geokey_weights:np.ndarray)-> np.ndarray:
    """assign total geogroup impressions proportionally to geokeys using Hagenbach-Bischoff quota
    :total_geogroup_impressions (int): Total number of impressions at the geogroup level to distribute
    :target(np.ndarray): numpy array with the geogroup to geokey target
    :geokey_weights (np.ndarray): numpy array of geokey weights
    :return: a numpy array of integer impressions for the geokey level breakdown
    """

    assert np.sum(geokey_weights)!=0, f"issue with geokey_weights sum to 0: {geokey_weights}"


    
    quota = sum(geokey_weights)/(1+total_geogroup_impressions)
    frac = [geoweight/quota for geoweight in geokey_weights]
    res = np.floor(frac).astype(int)

    n=total_geogroup_impressions-sum(res) #number of geokeys remaining to allocate

    if n==0: return res #done
    if n<0: return [min(x,total_geogroup_impressions) for x in res] #to handle case where geokey_weights=[0,0,..,1,0,...,0]

    #distribute the remaining impressions to the geokeys with the largest remainder
    remainders = frac - res
    
    #sort by the largest limit
    limit = remainders[np.argsort(remainders)[::-1][n-1]]

    #assign the remainder to the geokeys with the largest remainder 
    for i,r in enumerate(remainders):
        if r>=limit:
            res[i]+=1
            n-=1 # attempt to handle perfect equality
            if n==0: return res #done
    raise #should never happen


# Adapted D'Hondt method for running with geogroup to geokey target and larger impressions

In [13]:
import numpy as np 

def dhondt_distribute_geogroup_geokey( total_geogroup_impressions: int
                               ,target:np.ndarray
                               ,geokey_weights:np.ndarray
                               ,block_ratio:int=100)-> np.ndarray:
  """
  assign total geogroup impressions proportionally to geokeys using D'Hondt method
  :total_geogroup_impressions (int): Total number of impressions at the geogroup level to distribute
  :target(np.ndarray): numpy array with the geogroup to geokey target
  :geokey_weights (np.ndarray): numpy array of geokey weights
  :block_ratio (int): used to find the number of block impressions to subtract from the total 
   total_geogroup_impressions for each assignment to geokeys. Smaller values often lead to faster
   execution with less precision than larger values.
  :return: a numpy array of integer impressions for the geokey level breakdown
  """

  assert np.sum(geokey_weights)!=0, f"issue with geokey_weights sum to 0: {geokey_weights}"

  #replace any nan values in the weights
  geokey_weights[np.isnan(geokey_weights)] = 0
  #only use the targeted geokeys
  geokey_weights = target*geokey_weights

  #normalise geokey weights so they add to 100%
  geokey_weights /=geokey_weights.sum()
  #print(geokey_weights)


  number_geokeys = len(geokey_weights)
  final_allocations = np.zeros(number_geokeys,dtype=np.int64)

  #the number of block impressions to subtract from the total on each round.
  block_allocation = np.max((1,int(total_geogroup_impressions/block_ratio)))
  #print("block_allocation = ",block_allocation)
  left_impressions = total_geogroup_impressions

  while left_impressions > 0:
    
    quota= geokey_weights/(final_allocations+block_allocation)

    max_index = np.argmax(quota)

    #max_index = np.argsort(quota)[int((number_geokeys)/3)]
    final_allocations[max_index] +=block_allocation

    left_impressions -= block_allocation

    #if we have any excess truncate the last geokey allocation
    if left_impressions<0:
      final_allocations[max_index] += left_impressions

  #if there are no allocation for a geokey with positive weight, then 
  #take from the largest geokey and redistribute
  check_zero_alloc = np.where(final_allocations==0)
  #print("check_zero_alloc=", check_zero_alloc)
  #print("check_zero_alloc=", check_zero_alloc[0])

  if np.any(check_zero_alloc):
    for i in range(len(check_zero_alloc[0])):
      #if the rescaled geokey weights are positive and the final allocation is 0 we re-distribute
      #impressions from the largest geokey to the smallest ones.
      if geokey_weights[check_zero_alloc[0][i]] >0:
        #find the expected number of integer impressions for the zero allocated geokey
        redistribute_imp = np.round(total_geogroup_impressions * geokey_weights[check_zero_alloc[0][i]])
        #subtract the number of impressions that need to be redistributed from the largest geokey
        final_allocations[np.argmax(final_allocations)] -=  redistribute_imp
        #redistribute the impressions to the zero allocated geokey
        final_allocations[check_zero_alloc[0][i]]= redistribute_imp

    
  return final_allocations




# CVXPY solver + adapted d'hondt 

In [19]:
from os import truncate
import cvxpy as cp

def cvxpy_distribute_geogroup_geokey(total_geogroup_impressions: int
          ,target:np.ndarray
          ,geokey_weights:np.ndarray
          ,rounding_limit_fix:bool =True):
    """
    :total_geogroup_impressions (int): Total number of impressions at the geogroup level to distribute
    :target(np.ndarray): numpy array with the geogroup to geokey target
    :geokey_weights (np.ndarray): numpy array of geokey weights
    :rounding_limit_fix (bool): apply the adapted d'hondts method to fix rounding issues for the limits b
    :return: a numpy array of integer impressions for the geokey level breakdown
    """
    num_geokeys = len(geokey_weights)

    x = cp.Variable(num_geokeys, integer=True)

    c = cp.Constant(geokey_weights.astype(np.float64))

    #np.ones((num_geokeys,1))
    A = np.eye(num_geokeys)
    A = np.vstack((A,np.expand_dims(1-target,0)))

    if rounding_limit_fix:
      b =  hbischoff_distribute_geogroup_base(total_geogroup_impressions,geokey_weights)
    else:
      b = total_geogroup_impressions * geokey_weights

    b =  np.hstack((b,0))
    #print("b=",b)
    objective = cp.Maximize(c.T@x)
    
    constraints = [A @ x <= b]

    problem = cp.Problem(objective, constraints)

    problem.solve(verbose=False,solver='ECOS_BB')

    if x.value is None:
        return None, None
    
    return x.value


total_geogroup_impressions = 3000000
target  = np.array([1,1,1,1,1,1])

geokey_weights  = np.array([0.1,0.2,0.1,0.2,0.3,0.001])

print(np.vstack((np.eye(6), np.expand_dims(np.array(1-target),axis=0))))

out = cvxpy_distribute_geogroup_geokey(total_geogroup_impressions
                                 ,target
                                 ,geokey_weights
                                 ,True)

print("check output reconstructed out = ", out)

print("check output reconstructed sum out = ", np.sum(out))

print("check output reconstructed weights = ", np.sum(out/sum(out)))

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]]
check output reconstructed out =  [332962.0001035  665926.99992964 332962.0001035  665926.99992964
 998886.99999048 -32826.99995133]
check output reconstructed sum out =  2963838.000105447
check output reconstructed weights =  1.0000000000000002


# Test on dummy data

In [18]:
#Assumed total geogroup impressions that need to be distributed
total_geogroup_impressions = 3000000
total_geogroup_impressions = 9000  
#Assume we target all geokeys
target  = np.array([1]*9)
#Assume weight for the distribution across geokeys
#geokey_weights  = np.array([0.1,0.2,0.1,0.2,0.3,0.1])
#geokey_weights  = np.array([0.1]*4 +[0.599]+[0.001])
#geokey_weights  = np.array([1/6]*6)
geokey_weights  = np.array([1/9]*9,dtype=np.float64)

#print("check  = " , geokey_weights*(total_geogroup_impressions+1))
#print("check  = " , geokey_weights/(sum(geokey_weights)/(1+total_geogroup_impressions)))
#print("check  = " ,   [geoweight/(sum(geokey_weights)/(1+total_geogroup_impressions)) for geoweight in geokey_weights])
#print("check =",np.divide(geokey_weights,np.divide(sum(geokey_weights),(1+total_geogroup_impressions))))

#d'hondt method
final_allocations_dhondt = dhondt_distribute_geogroup_geokey(total_geogroup_impressions
                                                    ,target
                                                    ,geokey_weights)

#Hbischof method
final_allocations_hbischoff = hbischoff_distribute_geogroup_geokey(total_geogroup_impressions
                                                    ,target
                                                    ,geokey_weights)


#cvxpy method
final_allocations_cvxpy = cvxpy_distribute_geogroup_geokey(total_geogroup_impressions
                                                    ,target
                                                    ,geokey_weights
                                                    ,True)


print("Check to see final hdondt allocations sum to the total geogroup impressions = ",np.sum(final_allocations_dhondt)==total_geogroup_impressions)
print("Check to see final hbischoff allocations sum to the total geogroup impressions = ",np.sum(final_allocations_hbischoff)==total_geogroup_impressions)
print("Check to see if the geokey_weights sum to 1 = ",np.sum(geokey_weights))
print("Check the theoretical geokey weights = ",geokey_weights)
print("Check the allocated geokey weights dhondt = ", (final_allocations_dhondt/np.sum(final_allocations_dhondt)))
print("Check the allocated geokey dhondt = ", final_allocations_dhondt)
print("Check the allocated geokey weights hbischoff = ", (final_allocations_hbischoff/np.sum(final_allocations_hbischoff)))
print("Check the allocated geokey hbischoff = ", final_allocations_hbischoff)
print("Check the allocated geokey hbischoff sum = ", np.sum(final_allocations_hbischoff))
print("Check the allocated geokey weights cvxpy = ", (final_allocations_cvxpy/np.sum(final_allocations_cvxpy)))
print("Check the allocated geokey cvxpy = ", final_allocations_cvxpy)
print("Check the allocated geokey cvxpy sum = ", np.sum(final_allocations_cvxpy))



Check to see final hdondt allocations sum to the total geogroup impressions =  True
Check to see final hbischoff allocations sum to the total geogroup impressions =  True
Check to see if the geokey_weights sum to 1 =  1.0
Check the theoretical geokey weights =  [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
Check the allocated geokey weights dhondt =  [0.12 0.11 0.11 0.11 0.11 0.11 0.11 0.11 0.11]
Check the allocated geokey dhondt =  [1080  990  990  990  990  990  990  990  990]
Check the allocated geokey weights hbischoff =  [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
Check the allocated geokey hbischoff =  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Check the allocated geokey hbischoff sum =  9000
Check the allocated geokey weights cvxpy =  [0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111]
Check the allocated 